In [1]:
### Probabilistic Attempt with Pangu Weather (!! Actually not!! )
### For the ML based probabilistic models that WeatherBench 2 has (Pangu, Fuxi), it doesn't have ensemble forecasts for them?
### Only ensemble forecaster to test on is the ECMWF IFS ENS model (Physical numerically based)


In [2]:
import apache_beam

In [3]:
import os
import weatherbench2
import xarray as xr
import numpy as np
import math
from weatherbench2.regions import SliceRegion, ExtraTropicalRegion
from weatherbench2.evaluation import evaluate_in_memory
from weatherbench2 import config
from weatherbench2.metrics import CRPS, CRPSSpread, CRPSSkill, EnsembleVariance, EnsembleMeanMSE, EnsembleIgnoranceScore, EnsembleBrierScore, EnergyScore, EnergyScoreSkill, EnergyScoreSpread, RankHistogram 

In [4]:
forecast_path = 'gs://weatherbench2/datasets/ifs_ens/2018-2022-64x32_equiangular_conservative.zarr'
obs_path = 'gs://weatherbench2/datasets/era5/1959-2022-6h-64x32_equiangular_conservative.zarr'
climatology_path = 'gs://weatherbench2/datasets/era5-hourly-climatology/1990-2019_6h_64x32_equiangular_conservative.zarr'

In [5]:
climatology = xr.open_zarr(climatology_path)


In [6]:
forecasts = xr.open_zarr(forecast_path)
observations = xr.open_zarr(obs_path)


In [6]:
test = forecasts['geopotential'].sel(level = 500, time = slice('2020-01-01', '2020-12-31'))
ensembledim = test[0,:,0,0,0].size

In [7]:
paths1 = config.Paths(
    forecast=forecast_path,
    obs=obs_path,
    output_dir='./',   # Directory to save evaluation results
)

selection1 = config.Selection(
    variables=[
        'geopotential',
    ],
    levels=[500],
    time_slice=slice('2020-01-01', '2020-2-01'),
)

data_config1 = config.Data(selection=selection1, paths=paths1)

regions1 = {
    'global': SliceRegion(),
}

eval_configs1 = {
  'MeanMSEVariance': config.Eval(
      metrics={
          #'CRPS': CRPS('number'),
          #'CRPSSpread': CRPSSpread('number'),
          #'CRPSSkill': CRPSSkill('number'),
          'EnsembleVariance': EnsembleVariance('number'),
          'EnsembleMeanMSE': EnsembleMeanMSE('number'),
          #'EnsembleIgnoranceScore': EnsembleIgnoranceScore('number'),
          #'EnsembleBrierScore': EnsembleBrierScore('number'),
          #'EnergyScore': EnergyScore('number'),
          #'EnergyScoreSkill': EnergyScoreSkill('number'),
          #'EnergyScoreSpread': EnergyScoreSpread('number'),
          #'RankHistogram': RankHistogram('number', 40),
      },
      regions=regions1
  )
}


evaluate_in_memory(data_config1, eval_configs1)



In [9]:
full = xr.open_dataset('./OnlyEnergyScore.nc')
mini = xr.open_dataset('./OnlyEnergyScoreMini.nc')
crpsmini = xr.open_dataset('./OnlyCRPSMini.nc')
spreadskill = xr.open_dataset('./SpreadandSkill.nc')
meanMSEvariance = xr.open_dataset('./MeanMSEVariance.nc')

In [17]:
mini['geopotential'][:,:,:,:].values
full['geopotential'][:,:,:,:].values

array([[[[ 16.40059168],
         [ 17.37380748],
         [ 21.90202082],
         [ 22.90374028],
         [ 28.63360494],
         [ 31.81376322],
         [ 39.01819658],
         [ 44.04651572],
         [ 51.68230127],
         [ 57.91323458],
         [ 67.29016766],
         [ 74.74342197],
         [ 85.50607702],
         [ 94.58680874],
         [106.54174907],
         [116.78540843],
         [130.36353479],
         [141.70619667],
         [156.08422899],
         [168.37324771],
         [183.27989723],
         [195.86959851],
         [211.10127055],
         [224.03251049],
         [239.35082032],
         [252.27782218],
         [267.28129823],
         [279.85692916],
         [294.38048833],
         [306.46043771],
         [320.08435552],
         [331.38892106],
         [344.09150722],
         [354.43868065],
         [366.01286461],
         [375.52007359],
         [386.0933858 ],
         [394.6009482 ],
         [404.27010527],
         [411.86879601],


In [21]:
crpsmini['geopotential'][:,:,:,:].values

array([[[[ 15.39833045],
         [ 13.90813641],
         [ 18.48164672],
         [ 18.79502823],
         [ 23.33586116],
         [ 25.09675912],
         [ 29.79921545],
         [ 32.66286534],
         [ 37.49048854],
         [ 40.74957466],
         [ 46.30516335],
         [ 50.02635873],
         [ 56.65534858],
         [ 61.21588492],
         [ 69.04774343],
         [ 73.62024404],
         [ 82.33203342],
         [ 87.12597117],
         [ 96.89399812],
         [101.96509481],
         [112.14784674],
         [117.16877823],
         [127.53752385],
         [133.17227312],
         [143.99448775],
         [149.5352887 ],
         [160.33624455],
         [166.31286538],
         [176.7369237 ],
         [182.8505406 ],
         [192.72160336],
         [198.78255656],
         [208.19766894],
         [214.02217448],
         [222.52567799],
         [228.39327023],
         [236.27860973],
         [241.77518914],
         [248.94896831],
         [254.17263142],


In [8]:
skill = spreadskill['geopotential'].sel(metric = 'CRPSSkill')
spread = spreadskill['geopotential'].sel(metric = 'CRPSSpread')

#print(skill.values-spread/2)
print(skill.values)
spread.values

NameError: name 'spreadskill' is not defined

In [24]:
latitude = forecasts['latitude'][:].values
delta = 2.8125
theta_upper = latitude + delta
theta_lower = latitude - delta

# Calculate weights based on the provided formula
weights = (np.sin(np.radians(theta_upper)) - np.sin(np.radians(theta_lower)))
weights /= weights.sum()
weights *= 32

weights

array([0.07704437, 0.23039114, 0.38151911, 0.52897285, 0.67133229,
       0.80722643, 0.93534654, 1.05445875, 1.16341595, 1.26116882,
       1.34677594, 1.41941287, 1.47838008, 1.52310968, 1.55317091,
       1.56827425, 1.56827425, 1.55317091, 1.52310968, 1.47838008,
       1.41941287, 1.34677594, 1.26116882, 1.16341595, 1.05445875,
       0.93534654, 0.80722643, 0.67133229, 0.52897285, 0.38151911,
       0.23039114, 0.07704437])

In [ ]:
#Testing on Mini, code not generalised

#15 extra days lead
a = forecasts['geopotential'].sel(level = 500, time = slice('2020-01-01', '2020-02-01'))[:,:,:,:]
b = observations['geopotential'].sel(level=500, time = slice('2020-01-01', '2020-02-16'))[:,:,:]

#Skill
skills = []
for i in range(61):
    ashifted = a.assign_coords(time = a.time + a[:,:,i,:,:]['prediction_timedelta'].values)
    if (i ==60):
        vals = np.sum(abs(ashifted[:,:,i,:,:]-b[i:,:,:][::2,:,:]) *weights[None,None,:], axis = (0,1,2,3))/(64*50*64*32)
    else:
        vals = np.sum(abs(ashifted[:,:,i,:,:]-b[i:i-60,:,:][::2,:,:]) *weights[None,None,:], axis = (0,1,2,3))/(64*50*64*32)
    skills.append(vals.values)

skills


[array(23.73866304),
 array(23.90021249),
 array(30.91322866),
 array(34.05420002),
 array(42.00982705),
 array(47.46495631),
 array(56.17729284),
 array(63.13925993),
 array(72.27117911),
 array(80.00750331),
 array(90.22649833),
 array(98.77403801),
 array(110.44833038),
 array(120.39775749),
 array(133.85186204),
 array(144.30469092),
 array(159.08652681),
 array(170.19169643),
 array(186.51491313),
 array(198.39449838),
 array(215.58971091),
 array(227.77626475),
 array(245.47868984),
 array(258.54356644),
 array(276.92603697),
 array(290.12438084),
 array(308.71052315),
 array(322.39835109),
 array(340.54700529),
 array(354.34626371),
 array(371.89054462),
 array(385.48179658),
 array(402.24125224),
 array(415.18946574),
 array(430.66399708),
 array(443.29428712),
 array(457.68082199),
 array(469.42743805),
 array(482.65964672),
 array(493.65758205),
 array(505.7610166),
 array(515.75075474),
 array(526.55196797),
 array(535.46755875),
 array(545.40918452),
 array(553.58284313),
 

In [ ]:
#Page 12 of WeatherBench 2 paper suggests more efficient way to calculate E[X-X'], using loops, not done here. 
#Spread 

Spreads = []

for i in range(61):
    ashifted = a.assign_coords(time = a.time + a[:,:,i,:,:]['prediction_timedelta'].values)
    if (i ==60):
        vals = np.sum(abs(ashifted[:,:,i,:,:]-b[i:,:,:][::2,:,:]) *weights[None,None,:], axis = (0,1,2,3))/(64*50*64*32)
    else:
        vals = np.sum(abs(ashifted[:,:,i,:,:]-b[i:i-60,:,:][::2,:,:]) *weights[None,None,:], axis = (0,1,2,3))/(64*50*64*32)
    Spreads.append(vals.values)

skills

In [ ]:
#Page 12 of WeatherBench 2 paper suggests more efficient way to calculate E[X-X'], using loops, not done here. 
#However, this loop method for just lag 0 takes 6 minutes.
#
# Spread 
#

a = forecasts['geopotential'].sel(level = 500, time = slice('2020-01-01', '2020-02-01'))[:,:,:,:]
weighted_abs_differences = xr.zeros_like((abs(a[:, 0,0, :, :] - a[:, 1,0, :, :]) * weights[None,None,:]))

for m in range(50):
    for n in range(50):
        if m != n:
            weighted_abs = abs(a[:, m,0, :, :] - a[:, n,0, :, :]) * weights[None,None,:]
            weighted_abs_differences += weighted_abs


final_metric = np.sum(weighted_abs_differences) / (50*(50 - 1)*64*64*32)  


In [ ]:
#Takes ~6 Minutes to run, but does calculate spread of 1 lag correctly... 
final_metric.values

array(16.68066519)

In [12]:
EnsembleMSE = meanMSEvariance['geopotential'].sel(metric = 'EnsembleMeanMSE')
EnsembleMSE.values

array([[[   643.19363725],
        [   600.35705019],
        [  1042.54701625],
        [  1156.03471334],
        [  1831.03147814],
        [  2204.00481844],
        [  3209.64001283],
        [  4020.27483862],
        [  5483.44758881],
        [  6767.04246415],
        [  8989.896634  ],
        [ 11000.07785754],
        [ 14227.08634379],
        [ 17309.55426818],
        [ 21976.23598337],
        [ 26185.70819817],
        [ 32403.09514525],
        [ 38011.80586588],
        [ 46071.81915807],
        [ 53188.02243701],
        [ 62883.96451214],
        [ 71277.23580635],
        [ 82793.46937047],
        [ 93155.89817985],
        [106728.72008096],
        [118501.07147269],
        [133628.14539764],
        [146940.6450115 ],
        [163364.5850061 ],
        [177647.38303384],
        [194691.04140257],
        [209334.603585  ],
        [226589.68576453],
        [241510.93222677],
        [258741.36311211],
        [274139.07817801],
        [291259.24099161],
 

In [78]:
a = forecasts['geopotential'].sel(level = 500, time = slice('2020-01-01', '2020-02-01'))[:,:,:,:]
b = observations['geopotential'].sel(level=500, time = slice('2020-01-01', '2020-02-16'))[:,:,:]

#Take ensemble mean first


MeanMSEs = []
for i in range(61):
    ashifted = a.assign_coords(time = a.time + a[:,:,i,:,:]['prediction_timedelta'].values)
    if (i ==60):
        vals = np.sum(((np.sum(ashifted[:,:,i,:,:],axis=1)/50-b[i:,:,:][::2,:,:])**2) *weights[None,None,:], axis = (0,1,2))/(64*64*32)
    else:
        vals = np.sum(((np.sum(ashifted[:,:,i,:,:],axis=1)/50-b[i:i-60,:,:][::2,:,:])**2) *weights[None,None,:], axis = (0,1,2))/(64*64*32)
    val = vals.values
    #print(val)
    MeanMSEs.append(val)

MeanMSEs

[array(643.19363725),
 array(600.35705019),
 array(1042.54701625),
 array(1156.03471334),
 array(1831.03147814),
 array(2204.00481844),
 array(3209.64001283),
 array(4020.27483862),
 array(5483.44758881),
 array(6767.04246415),
 array(8989.896634),
 array(11000.07785754),
 array(14227.08634379),
 array(17309.55426818),
 array(21976.23598337),
 array(26185.70819817),
 array(32403.09514525),
 array(38011.80586588),
 array(46071.81915807),
 array(53188.02243701),
 array(62883.96451214),
 array(71277.23580635),
 array(82793.46937047),
 array(93155.89817985),
 array(106728.72008096),
 array(118501.07147269),
 array(133628.14539764),
 array(146940.6450115),
 array(163364.5850061),
 array(177647.38303384),
 array(194691.04140257),
 array(209334.603585),
 array(226589.68576453),
 array(241510.93222677),
 array(258741.36311211),
 array(274139.07817801),
 array(291259.24099161),
 array(306395.50650631),
 array(322913.00520517),
 array(337195.46402956),
 array(352458.01821256),
 array(365079.4329

In [74]:
np.sum(((np.sum(a[:,:,0,:,:],axis=1)/50-b[0:-60,:,:][::2,:,:])**2)*weights[None,None,:],axis=(0,1,2)).values/(64*64*32)

643.1936372523292

In [75]:
a

<xarray.DataArray 'geopotential' (time: 64, number: 50,
                                  prediction_timedelta: 61, longitude: 64,
                                  latitude: 32)>
dask.array<getitem, shape=(64, 50, 61, 64, 32), dtype=float32, chunksize=(4, 50, 1, 64, 32), chunktype=numpy.ndarray>
Coordinates:
  * latitude              (latitude) float64 -87.19 -81.56 ... 81.56 87.19
    level                 int32 500
  * longitude             (longitude) float64 0.0 5.625 11.25 ... 348.8 354.4
  * number                (number) int32 1 2 3 4 5 6 7 ... 44 45 46 47 48 49 50
  * prediction_timedelta  (prediction_timedelta) timedelta64[ns] 00:00:00 ......
  * time                  (time) datetime64[ns] 2020-01-01 ... 2020-02-01T12:...
Attributes:
    long_name:      Geopotential
    short_name:     z
    standard_name:  geopotential
    units:          m**2 s**-2

In [ ]:
skills = []
for i in range(61):
    ashifted = a.assign_coords(time = a.time + a[:,:,i,:,:]['prediction_timedelta'].values)
    if (i ==60):
        vals = np.sum(abs(ashifted[:,:,i,:,:]-b[i:,:,:][::2,:,:]) *weights[None,None,:], axis = (0,1,2,3))/(64*50*64*32)
    else:
        vals = np.sum(abs(ashifted[:,:,i,:,:]-b[i:i-60,:,:][::2,:,:]) *weights[None,None,:], axis = (0,1,2,3))/(64*50*64*32)
    skills.append(vals.values)

skills

In [ ]:
#Have had some work with probabilistic diagnostics, but time demands are increasing. Going to try experimenting with sigkernel now